### 1a. Commenting Variables and Steps

Here is a revised version of the code with comments explaining each variable and specifying which are local or global:

In [556]:
# Global variable: List of RNA sequences to analyze
rna_sequences = [
    'GGGCUAGCGUAUCGAUGCUA', 'CGCUAUAAAUGCGUAGGUC', 'CUAGCGGUAAUCGCAUUA', 
    'AUGCUAGGCUAUUAGCGA', 'UAGGCAUAAAGGCUAUGC', 'CUAGCGGUAAUCGCAUU', 
    'GCUAUGCAGGCAAUUGCUA', 'GGGCCGUAUCGAUGCUAGCGUAC', 'AGGCGAUAUAAACGCUUA', 'GGGCGGUCAUUAACGGAUUUGA'
]

# Global variables: Each treatment and control list contains expression counts of each gene under respective conditions
treatment_1 = [120, 520, 180, 250, 500, 2, 300, 130, 530, 110] # List of expression counts for treatment 1
treatment_2 = [125, 510, 185, 245, 505, 0, 310, 135, 540, 115] # List of expression counts for treatment 2
treatment_3 = [118, 515, 175, 255, 495, 2, 295, 128, 525, 113] # List of expression counts for treatment 3

control_1 = [140, 290, 178, 240, 300, 0, 305, 150, 310, 135]    # List of expression counts for control 1
control_2 = [145, 285, 182, 248, 295, 0, 298, 148, 300, 137]    # List of expression counts for control 2
control_3 = [138, 288, 176, 252, 305, 3, 302, 152, 315, 132]    # List of expression counts for control 3

# Function to calculate average counts for each gene in the treatment group
def process_treatment_counts():
    total_treatment_counts = []  # Local variable: stores average counts per gene for treatment
    for i in range(len(rna_sequences)):
        # Local variable: average count of a gene across all treatment replicates
        average_count = (treatment_1[i] + treatment_2[i] + treatment_3[i]) / 3 
        total_treatment_counts.append(average_count)  # Append average to the list
    # Print average treatment counts with 2 decimal places
    print(f"Average treatment counts per RNA sequence: {[round(count, 2) for count in total_treatment_counts]}")

# Function to calculate average counts for each gene in the control group
def process_control_counts():
    total_control_counts = []  # Local variable: stores average counts per gene for control
    for i in range(len(rna_sequences)):
        # Local variable: average count of a gene across all control replicates
        average_count = (control_1[i] + control_2[i] + control_3[i]) / 3
        total_control_counts.append(average_count)  # Append average to the list
    # Print average control counts with 2 decimal places
    print(f"Average control counts per RNA sequence: {[round(count, 2) for count in total_control_counts]}")

# Function to calculate the average count across a single experiment (either control or treatment)
def calculate_average_count_per_experiment(expression_counts, experiment_type):
    total_counts = 0  # Local variable: stores the sum of counts in one experiment
    for count in expression_counts:
        total_counts += count  # Sum all counts in the experiment
    # Local variable: average count for the experiment
    average_count = total_counts / len(expression_counts)
    # Print average count with 2 decimal places
    print(f"Average count for {experiment_type} experiment: {round(average_count, 2)} total counts: {total_counts}")

# Run functions for processing treatment and control counts
process_treatment_counts()
process_control_counts()

# Calculate and print average counts for each control and treatment replicate
for control_experiment in [control_1, control_2, control_3]:
    calculate_average_count_per_experiment(control_experiment, 'control')
for treatment_experiment in [treatment_1, treatment_2, treatment_3]:
    calculate_average_count_per_experiment(treatment_experiment, 'treatment')


Average treatment counts per RNA sequence: [121.0, 515.0, 180.0, 250.0, 500.0, 1.33, 301.67, 131.0, 531.67, 112.67]
Average control counts per RNA sequence: [141.0, 287.67, 178.67, 246.67, 300.0, 1.0, 301.67, 150.0, 308.33, 134.67]
Average count for control experiment: 204.8 total counts: 2048
Average count for control experiment: 203.8 total counts: 2038
Average count for control experiment: 206.3 total counts: 2063
Average count for treatment experiment: 264.2 total counts: 2642
Average count for treatment experiment: 267.0 total counts: 2670
Average count for treatment experiment: 262.1 total counts: 2621


### 1c. Observing Data Anomalies

One of the sequences (index 5) has consistently very low counts in both control and treatment groups (0 or 2), which might indicate an error in sequencing, such as incomplete reads. This sequence may need to be examined further or excluded from analysis if it's an outlier.

Exercise 2: Comparing Sequences for Similarity
2a. Calculating Similarity Between Sequences

In [557]:
from difflib import SequenceMatcher

# Function to compare similarity between all sequences and print pairs with >90% similarity
def find_similar_sequences(sequences, threshold=0.9):
    for seq_1_index in range(len(sequences)):
        for seq_2_index in range(seq_1_index + 1, len(sequences)):
            similarity = SequenceMatcher(None, sequences[seq_1_index], sequences[seq_2_index]).ratio()
            if similarity > threshold:
                print(f"Sequences {seq_1_index} and {seq_2_index} are {similarity*100:.2f}% similar:\n  {sequences[seq_1_index]}\n  {sequences[seq_2_index]}\n")
    
find_similar_sequences(rna_sequences)


Sequences 2 and 5 are 97.14% similar:
  CUAGCGGUAAUCGCAUUA
  CUAGCGGUAAUCGCAUU



### 2b. List Comprehension (Optional)

You could use a list comprehension to generate the list of similar pairs instead of printing them directly.

In [558]:
def find_similar_sequences(sequences, threshold=0.9):
    similarity = [(rna_sequences[seq_1_index], rna_sequences[seq_2_index], SequenceMatcher(None, rna_sequences[seq_1_index], rna_sequences[seq_2_index]).ratio()) for seq_1_index in range(len(rna_sequences)) for seq_2_index in range(seq_1_index + 1, len(rna_sequences)) if SequenceMatcher(None, rna_sequences[seq_1_index], rna_sequences[seq_2_index]).ratio() > 0.9]
    if similarity:
        print(f"Similar sequences found: {similarity}")
find_similar_sequences(rna_sequences)

Similar sequences found: [('CUAGCGGUAAUCGCAUUA', 'CUAGCGGUAAUCGCAUU', 0.9714285714285714)]


### Exercise 3: Analyzing Expression Ratios Between Treatment and Control

In [559]:
# Calculate averages, ratios, and classify high/low changes in expression
def analyze_expression_changes():
    upregulated = []
    downregulated = []
    
    
    for seq_index in range(len(rna_sequences)):
        # Calculate average expression counts for treatment and control
        avg_treatment = (treatment_1[seq_index] + treatment_2[seq_index] + treatment_3[seq_index]) / 3
        avg_control = (control_1[seq_index] + control_2[seq_index] + control_3[seq_index]) / 3  

        fold_change = avg_treatment / avg_control
        
        # Threshold: Upregulation if fold change > 1.1, downregulation if < 0.9
        if fold_change and fold_change > 1.1:
            upregulated.append((rna_sequences[seq_index], fold_change))
        elif fold_change and fold_change < 0.9:
            downregulated.append((rna_sequences[seq_index], fold_change))
        # Print each gene's information with fold change rounded to 2 decimals
        print(f"Gene {seq_index} - Treatment: {avg_treatment:.2f}, Control: {avg_control:.2f}, Fold Change: {fold_change:.2f}")
    
    # Output up/down regulated genes
    print("\nUpregulated genes:", upregulated)
    print("Downregulated genes:", downregulated)

analyze_expression_changes()



Gene 0 - Treatment: 121.00, Control: 141.00, Fold Change: 0.86
Gene 1 - Treatment: 515.00, Control: 287.67, Fold Change: 1.79
Gene 2 - Treatment: 180.00, Control: 178.67, Fold Change: 1.01
Gene 3 - Treatment: 250.00, Control: 246.67, Fold Change: 1.01
Gene 4 - Treatment: 500.00, Control: 300.00, Fold Change: 1.67
Gene 5 - Treatment: 1.33, Control: 1.00, Fold Change: 1.33
Gene 6 - Treatment: 301.67, Control: 301.67, Fold Change: 1.00
Gene 7 - Treatment: 131.00, Control: 150.00, Fold Change: 0.87
Gene 8 - Treatment: 531.67, Control: 308.33, Fold Change: 1.72
Gene 9 - Treatment: 112.67, Control: 134.67, Fold Change: 0.84

Upregulated genes: [('CGCUAUAAAUGCGUAGGUC', 1.7902665121668597), ('UAGGCAUAAAGGCUAUGC', 1.6666666666666667), ('CUAGCGGUAAUCGCAUU', 1.3333333333333333), ('AGGCGAUAUAAACGCUUA', 1.7243243243243243)]
Downregulated genes: [('GGGCUAGCGUAUCGAUGCUA', 0.8581560283687943), ('GGGCCGUAUCGAUGCUAGCGUAC', 0.8733333333333333), ('GGGCGGUCAUUAACGGAUUUGA', 0.8366336633663367)]


### Exercise 4: Identifying Motifs Within Sequences
### 4a. Finding Sequences with Over 60% Similarity

We'll use the SequenceMatcher function to identify pairs of sequences with over 60% similarity, which could indicate the presence of shared motifs.

In [560]:
from difflib import SequenceMatcher

# Function to find sequences with >60% similarity
def find_similar_sequences_with_motifs(sequences, threshold=0.6):
    similar_sequences = []
    for seq_1_index in range(len(sequences)):
        for seq_2_index in range(seq_1_index + 1, len(sequences)):
            similarity = SequenceMatcher(None, sequences[seq_1_index], sequences[seq_2_index]).ratio()
            if similarity > threshold:
                print(f"Sequences {seq_1_index} and {seq_2_index} are {similarity*100:.2f}% similar:\n  {sequences[seq_1_index]}\n  {sequences[seq_2_index]}\n")
                similar_sequences.append((sequences[seq_1_index], sequences[seq_2_index], similarity))
    return similar_sequences

# Execute function and print results
similar_sequences = find_similar_sequences_with_motifs(rna_sequences)


Sequences 0 and 2 are 78.95% similar:
  GGGCUAGCGUAUCGAUGCUA
  CUAGCGGUAAUCGCAUUA

Sequences 0 and 5 are 75.68% similar:
  GGGCUAGCGUAUCGAUGCUA
  CUAGCGGUAAUCGCAUU

Sequences 0 and 6 are 71.79% similar:
  GGGCUAGCGUAUCGAUGCUA
  GCUAUGCAGGCAAUUGCUA

Sequences 0 and 7 are 79.07% similar:
  GGGCUAGCGUAUCGAUGCUA
  GGGCCGUAUCGAUGCUAGCGUAC

Sequences 0 and 9 are 61.90% similar:
  GGGCUAGCGUAUCGAUGCUA
  GGGCGGUCAUUAACGGAUUUGA

Sequences 1 and 6 are 68.42% similar:
  CGCUAUAAAUGCGUAGGUC
  GCUAUGCAGGCAAUUGCUA

Sequences 1 and 8 are 64.86% similar:
  CGCUAUAAAUGCGUAGGUC
  AGGCGAUAUAAACGCUUA

Sequences 2 and 5 are 97.14% similar:
  CUAGCGGUAAUCGCAUUA
  CUAGCGGUAAUCGCAUU

Sequences 2 and 6 are 75.68% similar:
  CUAGCGGUAAUCGCAUUA
  GCUAUGCAGGCAAUUGCUA

Sequences 2 and 8 are 72.22% similar:
  CUAGCGGUAAUCGCAUUA
  AGGCGAUAUAAACGCUUA

Sequences 3 and 4 are 61.11% similar:
  AUGCUAGGCUAUUAGCGA
  UAGGCAUAAAGGCUAUGC

Sequences 4 and 8 are 72.22% similar:
  UAGGCAUAAAGGCUAUGC
  AGGCGAUAUAAACGCUUA

Sequen

4b. Searching for Motifs in a Specific Sequence

We'll take the sequence 'GGGCUAGCGUAUCGAUGCUA' and generate all possible substrings of length 4, then count if they occurr in the other RNA sequences.

In [561]:
# This solution is using dictionaries, while we have not worked with them yet but they will be covered in the next lesson
# Function to generate all substrings of a given length

def get_substrings(sequence, length):
    #initialize an empty list to store the substrings
    substrings = []
    #iterate over the sequence and get all substrings of the given length
    #we use len(sequence)-length+1 to make sure we don't go out of bounds
    for i in range(len(sequence)-length+1):
        #get the substring of the given length
        sub = sequence[i:i+length]
        #append the substring to the list
        substrings.append(sub)
    #make it a set to remove duplicates
    substrings = set(substrings)
    return substrings

# Define the target sequence and generate 4-nucleotide substrings
target_sequence = 'GGGCUAGCGUAUCGAUGCUA'

substrings = get_substrings(target_sequence, 4)

# Count occurrences of each substring in other sequences
def count_substring_occurrences(substrings, rna_sequences):
    motif_occurrences = {}
    #iterate over the motifs and the sequences
    for motif in substrings:
        for rna_sequence in rna_sequences:
            #check if the motif is in the sequence
            if motif in rna_sequence:
                #if it is, increment the count
                if motif in motif_occurrences:
                    motif_occurrences[motif] += 1
                else:
                    motif_occurrences[motif] = 1
    return motif_occurrences

# Get and print occurrences of each substring
print (substrings)
substring_occurrences = count_substring_occurrences(substrings, rna_sequences)

print("Occurrences of each 4-nucleotide substring in other sequences:")
for substring, count in substring_occurrences.items():
    print(f"{substring}: {count}")


{'UAGC', 'CGAU', 'GCUA', 'AUCG', 'CUAG', 'GAUG', 'UAUC', 'GGGC', 'UCGA', 'GCGU', 'UGCU', 'AUGC', 'GUAU', 'CGUA', 'GGCU', 'AGCG'}
Occurrences of each 4-nucleotide substring in other sequences:
UAGC: 5
CGAU: 3
GCUA: 6
AUCG: 4
CUAG: 5
GAUG: 2
UAUC: 2
GGGC: 3
UCGA: 2
GCGU: 3
UGCU: 4
AUGC: 6
GUAU: 2
CGUA: 3
GGCU: 3
AGCG: 5


4c. Repeating the Process for a Second Sequence

For the sequence 'CGCUAUAAAUGCGUAGGUC', we'll use 5-nucleotide substrings

In [562]:
# Define a new target sequence and generate 5-nucleotide substrings
target_sequence_2 = 'CGCUAUAAAUGCGUAGGUC'
motif5 = get_substrings(target_sequence_2, 5)

# Get and print occurrences of each 5-nucleotide substring in other sequences
motif5_occurrences = count_substring_occurrences(motif5, rna_sequences)
print("\nOccurrences of each 5-nucleotide substring in other sequences (CGCUAUAAAUGCGUAGGUC):")
for substring, count in motif5_occurrences.items():
    print(f"{substring}: {count}")



Occurrences of each 5-nucleotide substring in other sequences (CGCUAUAAAUGCGUAGGUC):
AAAUG: 1
CUAUA: 1
UAUAA: 2
CGUAG: 1
AUGCG: 1
AAUGC: 1
AGGUC: 1
UGCGU: 1
GCUAU: 4
CGCUA: 1
UAAAU: 1
GUAGG: 1
GCGUA: 3
AUAAA: 3
UAGGU: 1


Exercise 5: Compiling and Reporting Gene Expression Data with Motif Information

In this exercise, we’ll summarize the expression analysis data, including average counts, fold changes, and motif presence, then format and display it for clearer insights.
5a. Putting Everything Together

In [563]:
# Define a function to calculate fold change safely
def calculate_fold_change(treatment_avg, control_avg):
    return treatment_avg / control_avg if control_avg > 0 else None

# Compile and report the information
def compile_gene_expression_report():
    report = []
    #we choose the motifs to look for
    motif = 'AUAAA'
    for seq_1_index in range(len(rna_sequences)):
        sequence = rna_sequences[seq_1_index]
        avg_treatment = (treatment_1[seq_1_index] + treatment_2[seq_1_index] + treatment_3[seq_1_index]) / 3
        avg_control = (control_1[seq_1_index] + control_2[seq_1_index] + control_3[seq_1_index]) / 3
        fold_change = calculate_fold_change(avg_treatment, avg_control)
        if motif in sequence:
            motifs_report = motif
        else:
            motifs_report = 'None'
        # Add formatted data to report
        report.append((sequence, round(avg_control, 2), round(avg_treatment, 2), round(fold_change, 2), motifs_report))

    # Print the report in a formatted way
    # this is just an example done through trial and error, you can format it as you like
    # in the future lessons with pandas we will learn how to make better tables
    print("\nGene Expression Report:")
    print(f"{'Sequence'}\t\t|\t{'Avg Control'}\t|\t{'Avg Treatment'}\t|\t{'Fold Change'}\t|\t{'Motif'}\t|")
    print("=" * 112)
    for seq, avg_ctrl, avg_trt, fc, motif in report:
        print(f"{seq}\t|\t{avg_ctrl}\t\t|\t{avg_trt:}\t\t|\t{fc}\t\t|\t{motif}\t|")

# Run the function to print the report
compile_gene_expression_report()



Gene Expression Report:
Sequence		|	Avg Control	|	Avg Treatment	|	Fold Change	|	Motif	|
GGGCUAGCGUAUCGAUGCUA	|	141.0		|	121.0		|	0.86		|	None	|
CGCUAUAAAUGCGUAGGUC	|	287.67		|	515.0		|	1.79		|	AUAAA	|
CUAGCGGUAAUCGCAUUA	|	178.67		|	180.0		|	1.01		|	None	|
AUGCUAGGCUAUUAGCGA	|	246.67		|	250.0		|	1.01		|	None	|
UAGGCAUAAAGGCUAUGC	|	300.0		|	500.0		|	1.67		|	AUAAA	|
CUAGCGGUAAUCGCAUU	|	1.0		|	1.33		|	1.33		|	None	|
GCUAUGCAGGCAAUUGCUA	|	301.67		|	301.67		|	1.0		|	None	|
GGGCCGUAUCGAUGCUAGCGUAC	|	150.0		|	131.0		|	0.87		|	None	|
AGGCGAUAUAAACGCUUA	|	308.33		|	531.67		|	1.72		|	AUAAA	|
GGGCGGUCAUUAACGGAUUUGA	|	134.67		|	112.67		|	0.84		|	None	|


### Exercise 6: Reformat RNA Sequences into Codons

In [564]:
# Function to format sequences into codons separated by hyphens
def format_sequences_as_codons(sequences):
    formatted_sequences = []
    for sequence in sequences:
        codons = [sequence[i:i+3] for i in range(0, len(sequence), 3)]
        formatted_sequence = '-'.join(codons)
        formatted_sequences.append(formatted_sequence)
    return formatted_sequences

# Reformat and print each RNA sequence
formatted_sequences = format_sequences_as_codons(rna_sequences)
print("Formatted RNA sequences with codons:")
for seq in formatted_sequences:
    print(seq)


Formatted RNA sequences with codons:
GGG-CUA-GCG-UAU-CGA-UGC-UA
CGC-UAU-AAA-UGC-GUA-GGU-C
CUA-GCG-GUA-AUC-GCA-UUA
AUG-CUA-GGC-UAU-UAG-CGA
UAG-GCA-UAA-AGG-CUA-UGC
CUA-GCG-GUA-AUC-GCA-UU
GCU-AUG-CAG-GCA-AUU-GCU-A
GGG-CCG-UAU-CGA-UGC-UAG-CGU-AC
AGG-CGA-UAU-AAA-CGC-UUA
GGG-CGG-UCA-UUA-ACG-GAU-UUG-A
